In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-06-02 05:25:33.973959: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 05:25:34.080406: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-02 05:25:34.623660: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [2]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

In [3]:
import numpy as np
np.random.randint(0, 20)

6

In [5]:
def generate_split(fake_cols, file_path):

    
    loader = LJDataLoader(data_path=file_path)
    
    #GK: comment out sample cos shuffle = True which causes sampling to be redundant
    
    #loader.sample(0.1)
    loader.splitData()

    source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
    new_col_name = 'RandWaveFake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)
    
    source_architectures = ['RandWaveFake', 'ElevenLabs', 'UberDuck']
    new_col_name = 'Fake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

    #data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
    data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=fake_cols)
    
    train_count = data_df[data_df['type'] == 'train'].shape[0]
    dev_count = data_df[data_df['type'] == 'dev'].shape[0]
    test_count = data_df[data_df['type'] == 'test'].shape[0]

    print(f'# of Train instances: {train_count}')
    print(f'# of Dev instances: {dev_count}')
    print(f'# of Test instances: {test_count}')
    
    return data_df

In [6]:
loader = LJDataLoader(data_path='/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv')

In [7]:
df = generate_split(['ElevenLabs', 'UberDuck', 'Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow'], '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv')

# of Train instances: 78330
# of Dev instances: 26110
# of Test instances: 26110


In [8]:
df.shape

(130550, 6)

In [9]:
df.head()

,type,id,architecture,path,label,multiclass_label
0,train,LJ002-0233,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ030-0087,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ038-0289,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ028-0210,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ017-0001,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


In [43]:
df.columns

Index(['type', 'id', 'architecture', 'path', 'label', 'multiclass_label'], dtype='object')

In [44]:
df.type.value_counts()

train    78330
dev      26110
test     26110
Name: type, dtype: int64

In [10]:
df.architecture.value_counts()

Real                 13055
ElevenLabs           13055
UberDuck             13055
Full_Band_MelGan     13055
HifiGan              13055
MelGan               13055
MelGanLarge          13055
Multi_Band_MelGan    13055
Parallel_WaveGan     13055
Waveglow             13055
Name: architecture, dtype: int64

In [46]:
df.label.value_counts()

1    117495
0     13055
Name: label, dtype: int64

In [47]:
df.multiclass_label.value_counts()

0    13055
1    13055
2    13055
3    13055
4    13055
5    13055
6    13055
7    13055
8    13055
9    13055
Name: multiclass_label, dtype: int64

In [48]:
np.unique(df.id).shape[0]

13055

#### GK Testing

In [11]:
smile_manager = SmileFeatureManager(df)

100%|██████████| 10/10 [02:23<00:00, 14.39s/it]


In [15]:
smile_manager.feature_df

,type,id,architecture,path,label,multiclass_label,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,train,LJ002-0233,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,2.129408,0.028065,0.192024,0.243366,...,9.529998,0.560518,2.730733,2.732044,-20.000000,0.480135,121.310260,58.826866,128.822830,75.760080
1,train,LJ030-0087,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,1.924677,0.028571,0.294737,0.327967,...,7.537261,0.591446,2.773203,2.762229,19.942253,0.600441,114.922310,51.958990,117.646300,58.924770
2,train,LJ038-0289,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,2.188892,0.035714,0.234286,0.268195,...,8.763081,0.596172,2.720930,2.718240,19.999905,0.506655,126.037575,70.289215,119.342790,61.760730
3,train,LJ028-0210,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,3.001255,0.551689,0.309109,0.198558,...,8.494323,0.532472,2.763822,2.759283,19.998340,0.597494,124.486460,67.139920,119.512120,56.905670
4,train,LJ017-0001,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,2.507965,0.527370,0.680908,0.302279,...,9.178305,0.533144,2.650029,2.646701,19.999601,0.531791,115.491844,70.020120,126.838070,65.519050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130545,test,LJ021-0132,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9,1.760884,0.685298,0.535844,0.321625,...,8.502029,0.529507,2.719269,2.722790,-19.999527,0.607161,120.086670,65.445915,118.121230,62.161392
130546,test,LJ040-0053,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9,2.004604,0.598443,0.375973,0.289457,...,6.921431,0.428056,2.748192,2.759361,-19.937088,0.569690,133.381240,73.012240,116.486300,64.163400
130547,test,LJ040-0220,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9,1.682076,0.093418,0.377919,0.237795,...,8.218983,0.423514,2.882789,2.865987,19.819756,0.536003,131.601300,68.671745,112.601494,63.609764
130548,test,LJ038-0218,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9,2.485732,0.191489,0.142390,0.263357,...,5.224379,0.461306,2.336083,2.343161,-19.977005,0.672576,106.856670,47.224495,113.585480,54.351880


In [16]:
df

,type,id,architecture,path,label,multiclass_label
0,train,LJ002-0233,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ030-0087,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ038-0289,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ028-0210,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ017-0001,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
...,...,...,...,...,...,...
130545,test,LJ021-0132,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
130546,test,LJ040-0053,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
130547,test,LJ040-0220,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
130548,test,LJ038-0218,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9


In [17]:
os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary')
os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass')
    

Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audspec_lengthL1norm_sma_iqr1-3
2. audspec_lengthL1norm_sma_quartile1
3. audspec_lengthL1norm_sma_quartile2
4. audspec_lengthL1norm_sma_iqr2-3
5. audspec_lengthL1norm_sma_iqr1-2
6. audspec_lengthL1norm_sma_minPos
7. audspec_lengthL1norm_sma_maxPos
8. audspec_lengthL1norm_sma_quartile3
9. audspec_lengthL1norm_sma_range
10. audspec_lengthL1norm_sma_percentile1.0
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[11]_lpc1
2. jitterLocal_sma_percentile1.0
3. mfcc_sma_de[13]_lpgain
4. mfcc_sma[9]_lpc1
5. pcm_fftMag_spectralEntropy_sma_de_lpgain
6. jitterDDP_sma_quartile1
7. mfcc_sma[13]_lpgain
8. mfcc_sma[8]_lpc1
9. jitterDDP_sma_flatness
10. jitterDDP_sma_percentile1.0


In [ ]:
def generate_features(data_df, window_size, silence_threshold):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    cadence_manager = CadenceModelManager(data_df)
    cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline(window_size, silence_threshold) # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary')
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass')
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [ ]:
def train_eval(feature_store, fake_cols):
    results_cols = ['model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss']
    results = pd.DataFrame(columns=results_cols)
    
    for label_type in ['label', 'multiclass_label']:
        for k, v in feature_store.items():
            model_manager = ModelManager('decision_tree', v[0], v[1], merge_train_dev=True)
            model_manager.trainPredict(label_col=label_type)
            print(model_manager.class_accuracy)
            results = results.append(pd.DataFrame({'model':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value]}))
    
    return results
    
    

In [ ]:
def run(fake_cols, metadata_path, name, data_df=None, window_size=100, silence_threshold=0.1):
    if data_df is None:
        data_df = generate_split(fake_cols, metadata_path)
    feature_store = generate_features(data_df, window_size, silence_threshold)
    results = train_eval(feature_store, fake_cols)
    results.to_csv(f'/home/ubuntu/data/results/{name}.csv', index=False)
    
    

In [ ]:
#file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
#run(['ElevenLabs'], file_path, '16KHz_ElevenLabs')

For the columns, we want:
- Which dataset are we using
- What are the fake cols
- Binary/Multi-Class
- Feature Generation Method
- Accuracy
- Loss

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['UberDuck'], file_path, '16KHz_UberDuck', window_size=328, silence_threshold=0.01491875)

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['UberDuck', 'ElevenLabs'], file_path, '16KHz_ElevenLabs_and_UberDuck')

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['ElevenLabs', 'UberDuck', 'RandWaveFake'], file_path, '16KHz_Mix')

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['ElevenLabs'], file_path, '16KHz_ElevenLabs_Laundered')

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['UberDuck'], file_path, '16KHz_UberDuck_Laundered')

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['UberDuck', 'ElevenLabs'], file_path, '16KHz_ElevenLabs_and_UberDuck_Laundered')

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['ElevenLabs', 'UberDuck', 'RandWaveFake'], file_path, '16KHz_Mix_Laundered')